<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/CIFAR10_train_test/blob/main/CNN_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchsummary

**Import requirements**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary

**Load data and define hyperparameters**

In [34]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load our dataset
train_data = datasets.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_data, batch_size=5, shuffle=True)
test_data = datasets.CIFAR10(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=True)

# hyperparameters
learning_rate = 0.002
batch_size = 32
num_epochs = 50

Files already downloaded and verified
Files already downloaded and verified


**Implement CNN model**

In [39]:
# our CNN model with three Layers and Relu as an activation function
class CNN(nn.Module):
  def __init__(self, channels = 3, num_classes = 10):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.flat = nn.Flatten()
    self.linear = nn.Linear(128*4*4, num_classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.flat(x)
    x = self.linear(x)
    x = self.softmax(x)
    return x

In [40]:
cnn_model = CNN().to(device)
summary(cnn_model, (3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 32, 16, 16]          --
|    └─Conv2d: 2-1                       [-1, 32, 32, 32]          896
|    └─ReLU: 2-2                         [-1, 32, 32, 32]          --
|    └─MaxPool2d: 2-3                    [-1, 32, 16, 16]          --
├─Sequential: 1-2                        [-1, 64, 8, 8]            --
|    └─Conv2d: 2-4                       [-1, 64, 16, 16]          18,496
|    └─ReLU: 2-5                         [-1, 64, 16, 16]          --
|    └─MaxPool2d: 2-6                    [-1, 64, 8, 8]            --
├─Sequential: 1-3                        [-1, 128, 4, 4]           --
|    └─Conv2d: 2-7                       [-1, 128, 8, 8]           73,856
|    └─ReLU: 2-8                         [-1, 128, 8, 8]           --
|    └─MaxPool2d: 2-9                    [-1, 128, 4, 4]           --
├─Flatten: 1-4                           [-1, 2048]                --
├─Line

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 32, 16, 16]          --
|    └─Conv2d: 2-1                       [-1, 32, 32, 32]          896
|    └─ReLU: 2-2                         [-1, 32, 32, 32]          --
|    └─MaxPool2d: 2-3                    [-1, 32, 16, 16]          --
├─Sequential: 1-2                        [-1, 64, 8, 8]            --
|    └─Conv2d: 2-4                       [-1, 64, 16, 16]          18,496
|    └─ReLU: 2-5                         [-1, 64, 16, 16]          --
|    └─MaxPool2d: 2-6                    [-1, 64, 8, 8]            --
├─Sequential: 1-3                        [-1, 128, 4, 4]           --
|    └─Conv2d: 2-7                       [-1, 128, 8, 8]           73,856
|    └─ReLU: 2-8                         [-1, 128, 8, 8]           --
|    └─MaxPool2d: 2-9                    [-1, 128, 4, 4]           --
├─Flatten: 1-4                           [-1, 2048]                --
├─Line

**Train data**

In [41]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

# train data
def train(model, train_loader, num_epochs):
  model.train()
  for epoch in range(num_epochs):
    print(f" Epoch: {epoch+1}")
    for batch_index, (data, targets) in enumerate(train_loader):
      data = data.to(device=device)
      targets = targets.to(device=device)
      output = model(data)
      loss = criterion(output, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"loss after this epoch: {loss.item()}")
    print("---------------------------------------------")

# check the acuuracy of the model
def checkaccuracy(model, loader):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on testing data")
  corrects = 0
  samples = 0
  model.eval()
  with torch.no_grad():
    for x, label in loader:
      x = x.to(device)
      label = label.to(device)
      outputs = model(x)
      _, predictions = outputs.max(1)
      corrects += (predictions == label).sum()
      samples += predictions.size(0)

  print(f'Got {corrects}/{samples} with the accuracy of {float(corrects)/float(samples)*100:.2f}')
  model.train()

In [42]:
train(cnn_model, train_loader, num_epochs)
checkaccuracy(cnn_model, test_loader)

 Epoch: 1
loss after this epoch: 1.8611485958099365
---------------------------------------------
 Epoch: 2
loss after this epoch: 2.4611496925354004
---------------------------------------------
 Epoch: 3
loss after this epoch: 1.8587656021118164
---------------------------------------------
 Epoch: 4
loss after this epoch: 2.059753179550171
---------------------------------------------
 Epoch: 5
loss after this epoch: 2.061150074005127
---------------------------------------------
 Epoch: 6
loss after this epoch: 2.061150074005127
---------------------------------------------
 Epoch: 7
loss after this epoch: 2.4611501693725586
---------------------------------------------
 Epoch: 8
loss after this epoch: 2.061150074005127
---------------------------------------------
 Epoch: 9
loss after this epoch: 2.2611501216888428
---------------------------------------------
 Epoch: 10
loss after this epoch: 2.061150074005127
---------------------------------------------
 Epoch: 11
loss after th